In [19]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import pandas

seed = 7
np.random.seed(seed)
def string2bits(s=''):
    return [bin(ord(x))[2:].zfill(8) for x in s]
def bits2string(b=None):
    return ''.join([chr(int(x, 2)) for x in b])
dataframe = pandas.read_csv("hash_md5_dataset_sample.csv", header=None)
dataset = dataframe.values
x = dataset[:, 0:2]
X = []
Y= []
x_test=[]
y_test=[]

for i in range(4000):
    X.append(((''.join(string2bits(x[i][0])))))

for i in range(4000):
    Y.append(((''.join(string2bits(x[i][1])))))

for i in range(4000,5000):
    x_test.append(((''.join(string2bits(x[i][0])))))

for i in range(4000,5000):
    y_test.append(((''.join(string2bits(x[i][1])))))


x_train = np.array([[int(x) for x in y] for y in X])
y_train = np.array([[int(x) for x in y] for y in Y])

x_test = np.array([[int(x) for x in y] for y in x_test])
y_test = np.array([[int(x) for x in y] for y in y_test])

model = Sequential()
model.add(Dense(256, input_dim=256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(8,activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()
model.fit(x_train, y_train,
          epochs=200,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=1024, verbose=1)
classes = model.predict(x_test, batch_size=128)
print(classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 4104      
Total params: 201,480
Trainable params: 201,480
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
4000/4000 [==============================] - 0s - loss: 0.3356 - acc: 0.8636     
Epoch 2/200
4000/4000 [==============================] - 0s - loss: 0.0843 - acc: 0.9866     
Epoch 3/200
4000/4000 [==============================] - 0s - loss: 0.0316 - acc

4000/4000 [==============================] - 0s - loss: 1.0926e-07 - acc: 1.0000     
Epoch 76/200
4000/4000 [==============================] - 0s - loss: 1.0953e-07 - acc: 1.0000     
Epoch 77/200
4000/4000 [==============================] - 0s - loss: 1.0936e-07 - acc: 1.0000     
Epoch 78/200
4000/4000 [==============================] - 0s - loss: 1.0966e-07 - acc: 1.0000     
Epoch 79/200
4000/4000 [==============================] - 0s - loss: 1.0933e-07 - acc: 1.0000     
Epoch 80/200
4000/4000 [==============================] - 0s - loss: 1.0935e-07 - acc: 1.0000     
Epoch 81/200
4000/4000 [==============================] - 0s - loss: 1.0940e-07 - acc: 1.0000     
Epoch 82/200
4000/4000 [==============================] - 0s - loss: 1.0921e-07 - acc: 1.0000     
Epoch 83/200
4000/4000 [==============================] - 0s - loss: 1.0978e-07 - acc: 1.0000     
Epoch 84/200
4000/4000 [==============================] - 0s - loss: 1.0940e-07 - acc: 1.0000     
Epoch 85/200
4000/4000 

4000/4000 [==============================] - 0s - loss: 1.0927e-07 - acc: 1.0000     
Epoch 157/200
4000/4000 [==============================] - 0s - loss: 1.1384e-07 - acc: 1.0000     
Epoch 158/200
4000/4000 [==============================] - 0s - loss: 1.1017e-07 - acc: 1.0000     
Epoch 159/200
4000/4000 [==============================] - 0s - loss: 1.0963e-07 - acc: 1.0000     
Epoch 160/200
4000/4000 [==============================] - 0s - loss: 1.0923e-07 - acc: 1.0000     
Epoch 161/200
4000/4000 [==============================] - 0s - loss: 1.0923e-07 - acc: 1.0000     
Epoch 162/200
4000/4000 [==============================] - 0s - loss: 1.0914e-07 - acc: 1.0000     
Epoch 163/200
4000/4000 [==============================] - 0s - loss: 1.0918e-07 - acc: 1.0000     
Epoch 164/200
4000/4000 [==============================] - 0s - loss: 1.0947e-07 - acc: 1.0000     
Epoch 165/200
4000/4000 [==============================] - 0s - loss: 1.0919e-07 - acc: 1.0000     
Epoch 166/200


In [5]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
out = model.predict_proba(x_test)
out = np.array(out)

threshold = np.arange(0.1,0.9,0.1)
acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])
print y_pred
print y_test

  32/1000 [..............................] - ETA: 0s[[0 1 1 ..., 1 0 1]
 [0 1 1 ..., 1 0 1]
 [0 1 0 ..., 0 0 0]
 ..., 
 [0 1 0 ..., 0 0 0]
 [0 1 1 ..., 0 0 1]
 [0 1 1 ..., 0 1 0]]
[[0 1 1 ..., 1 0 1]
 [0 1 1 ..., 1 0 1]
 [0 1 0 ..., 0 0 0]
 ..., 
 [0 1 0 ..., 0 0 0]
 [0 1 1 ..., 0 0 1]
 [0 1 1 ..., 0 1 0]]


In [6]:
print hamming_loss(y_test,y_pred)
total_correctly_predicted = len([i for i in range(len(y_test)) if (y_test[i]==y_pred[i]).sum() == 160])
print total_correctly_predicted/400. #exact accuracy for eg y_pred = [0,0,1,1,1] and y_test=[0,0,1,1,1]
print total_correctly_predicted

0.0
0.0
0


In [20]:
classes[classes > 0.5] = 1
classes[classes <= 0.5]=0
values=[]
classes[0]

array([ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.], dtype=float32)

In [41]:
values=[]
temp=''
for i in range(1000):
    for j in range(8):
        temp=temp+str(int(classes[i][j]))  
    values.append(temp)
    temp=''
#print values
print bits2string(values)

emXrMDcjMjgxT0aoBZ4dkfBHhh9iKKghXJzj7yL89tpEKa1uCo1NevGpD61b5GX6Yu6XGejnjXXNWEIieSPOquyuDXPxpy53LJAmTYwEJaR9sx4b7wtu8f12RPudJhLzLQa3TdY6hxOk1ZpW46xP4yXuAcyJZTshJknHNJI8zpiZgAUAoaa7LNfGuVtaA6W4JmE3Bj0YmAUw7aEMTImKq9xVOP0R8CQI4TJDXTCklwPi8tVgzM6sNrCA4cDRFSThiUrbv9VKC4rQPVdCeCDfOTEmqXCRfdn8WyoH2TsfgLNyABsytKV02Cbb4HB7YmSeL8CcSdxoep84knNY6oEP0QjQfj9hzD7kd8I6jqrZKoHh0I7kq5KU0UfYHzTDS30nzg36clFsiyxvkRcUAbDnTzLqeObiTSfkjaxNMl28Dg5QdJvQtEdtLBYTyPhwtwprlgEOLBM5W0AwxSQEGPav2z8uMrzCnMgTQcGU2EcWPUDOe5eAAKs100rJLSfiF8IhkNwLLUrRGezvVV7XXMNtZwF02W53x0SFNUxQRI3giQFZoQNiSonBqHSNnXYfgl8zUqghIK5OhzpCzO5bl9olUCkbwXq0gPI2fU57RhficgowRkdoShNK0EFVzNwBhBXZ1oSJ8HJtcPUgSzThrDfbbzEK2ja5wMimv3xKkoSuy8JhxfanrYjGs4dCIy980sXenvx3sAoChDTWbyj0WdVXrCzFoBBuDQaPXd34b2TLK2ykpBscDFBz2BDRZkiDoNXQHPwXUQ4qyrMdBjq2yYnB9KfrVLL8eBk3vXgsGO1KjyZwIMDejMYqI8JJWRG9XKMkgW6rDmwKGi8GDwgq6rWlFpipaBPzJqt5KTluNOguQO1mbf7EKd8n3kYr8Y82EFYltFSfzLqPnPcJFDvUs8iKEUesk2M4tnrtdNJqBt0cgTKZvrbqNim68KGKJGRFJnfiajiej2yLSSMLPXoEHGrosknH4JeZkfXgsIOyfHyb